In [1]:
# Step 2 — Verify environment (Workbench)

import sys, subprocess, os

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *pkgs])

# Install if needed (safe to run)
pip_install([
    "google-cloud-aiplatform",
    "kfp",
    "google-cloud-pipeline-components",
    "numpy<2",
])

# Basic imports check
import google.cloud.aiplatform as aiplatform
import kfp
from kfp import dsl
from kfp.dsl import component

print("Python:", sys.version.split()[0])
print("aiplatform:", aiplatform.__version__)
print("kfp:", kfp.__version__)

/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.cloud.aiplatform_v1beta1 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.aiplatform_v1beta1 past that date.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.cloud.aiplatform_v1 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.aiplatform_v1 past that date.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_supp

Python: 3.10.19
aiplatform: 1.138.0
kfp: 2.15.2


In [2]:
# Step 3 — Initialize Vertex AI SDK

from kfp import dsl, compiler
from kfp.dsl import component, Output, Input, Model, Metrics
from google.cloud import aiplatform
import time

from google_cloud_pipeline_components.v1.custom_job import create_custom_training_job_from_component
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import ModelDeployOp
from google_cloud_pipeline_components.types import artifact_types

# Fill these in:
PROJECT_ID = "vertex-ai-487907"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://vertex-mlops-vinzur"
PIPELINE_ROOT = f"{STAGING_BUCKET}/phase3-custom-container"
SERVICE_ACCOUNT = "vertex-pipeline-sa@vertex-ai-487907.iam.gserviceaccount.com"

TRAIN_IMAGE_URI = "us-central1-docker.pkg.dev/vertex-ai-487907/vertex-mlops/train-sklearn:1"
SERVE_IMAGE_URI = "us-central1-docker.pkg.dev/vertex-ai-487907/vertex-mlops/serve-sklearn:1"

ENDPOINT_RESOURCE_NAME = "projects/.../locations/us-central1/endpoints/..."

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)


In [3]:
# Container-based training component (runs your training image)

from kfp import dsl
from kfp.dsl import Output, Model
from google_cloud_pipeline_components.v1.custom_job import create_custom_training_job_from_component

@dsl.container_component
def train_container(n_rows: int, model: Output[Model]):
    return dsl.ContainerSpec(
        image=TRAIN_IMAGE_URI,
        args=[
            "--n_rows", n_rows,
            "--model_dir", model.path,
        ],
    )
    
CustomTrainOp = create_custom_training_job_from_component(
    component_spec=train_container,
    display_name="phase3-custom-container-train",
    machine_type="n1-standard-4",
    replica_count=1,
    base_output_directory=PIPELINE_ROOT,
    service_account=SERVICE_ACCOUNT,
)

In [4]:
# Evaluation component (reads model artifact, logs accuracy, returns accuracy)

from typing import NamedTuple
from kfp.dsl import component, Input, Output, Model, Metrics

@component(
    base_image="python:3.10-slim",
    packages_to_install=["numpy<2", "pandas", "scikit-learn", "joblib"],
)

def evaluate(model: Input[Model], metrics: Output[Metrics]) -> NamedTuple("Outputs", [("accuracy", float)]):
    import joblib
    import numpy as np
    import pandas as pd
    from sklearn.metrics import accuracy_score
    import os

    print("Model path mounted at:", model.path)
    print("Files in model dir:", os.listdir(model.path))
    clf = joblib.load(f"{model.path}/model.joblib")

    rng = np.random.default_rng(123)
    n = 300
    x1 = rng.normal(size=n)
    x2 = rng.normal(size=n)
    y = (x1 + 0.5 * x2 + rng.normal(scale=0.3, size=n) > 0).astype(int)

    df = pd.DataFrame({"x1": x1, "x2": x2, "y": y})
    X = df[["x1", "x2"]]
    y_true = df["y"]

    preds = clf.predict(X)
    acc = float(accuracy_score(y_true, preds))

    metrics.log_metric("accuracy", acc)
    print("accuracy:", acc)

    return (acc,)

In [5]:
from kfp.dsl import component, Input, Model
from typing import NamedTuple

@component(
    base_image="python:3.10-slim",
    packages_to_install=["google-cloud-storage"],
)
def export_model_to_gcs(
    model: Input[Model],
    export_gcs_dir: str,
) -> NamedTuple("Outputs", [("exported_uri", str)]):
    import os
    from google.cloud import storage

    if not export_gcs_dir.startswith("gs://"):
        raise ValueError(f"export_gcs_dir must be gs://..., got: {export_gcs_dir}")

    # Parse gs://bucket/prefix
    path = export_gcs_dir[5:]
    bucket_name, _, prefix = path.partition("/")
    prefix = prefix.rstrip("/")

    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Upload everything in the model artifact directory
    local_dir = model.path
    files = []
    for root, _, fs in os.walk(local_dir):
        for f in fs:
            files.append(os.path.join(root, f))

    if not files:
        raise RuntimeError(f"No files found in model.path: {local_dir}")

    for local_path in files:
        rel = os.path.relpath(local_path, local_dir).replace("\\", "/")
        blob_path = f"{prefix}/{rel}" if prefix else rel
        bucket.blob(blob_path).upload_from_filename(local_path)

    # Sanity check: must have model.joblib at least
    # (If you saved it elsewhere, adjust this)
    check_blob = bucket.blob(f"{prefix}/model.joblib")
    if not check_blob.exists(client):
        raise RuntimeError(f"Upload done but model.joblib not found at {export_gcs_dir}/model.joblib")

    return (export_gcs_dir,)

In [6]:
# Pipeline (gate + upload model + deploy to existing endpoint)

from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import ModelDeployOp
from google_cloud_pipeline_components.types import artifact_types
from kfp import dsl

@dsl.pipeline(name="phase3-custom-container-single-endpoint")
def phase3_pipeline(n_rows: int = 500, min_accuracy: float = 0.80):

    train_task = CustomTrainOp(n_rows=n_rows)
    eval_task = evaluate(model=train_task.outputs["model"])

    with dsl.If(eval_task.outputs["accuracy"] >= min_accuracy, name="deploy_if_good"):

        # Deterministic export location (no guessing Vertex internals)
        export_gcs_dir = f"{PIPELINE_ROOT}/exported-models/{dsl.PIPELINE_JOB_ID_PLACEHOLDER}"

        export_task = export_model_to_gcs(
            model=train_task.outputs["model"],
            export_gcs_dir=export_gcs_dir,
        )

        unmanaged = dsl.importer(
            artifact_uri=export_task.outputs["exported_uri"],
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={"containerSpec": {"imageUri": SERVE_IMAGE_URI}},
        )

        uploaded = ModelUploadOp(
            project=PROJECT_ID,
            location=LOCATION,
            display_name="phase3-sklearn-model",
            unmanaged_container_model=unmanaged.outputs["artifact"],
        )

        endpoint_art = dsl.importer(
            artifact_uri=ENDPOINT_RESOURCE_NAME,
            artifact_class=artifact_types.VertexEndpoint,
            metadata={"resourceName": ENDPOINT_RESOURCE_NAME},
        )

        ModelDeployOp(
            model=uploaded.outputs["model"],
            endpoint=endpoint_art.outputs["artifact"],
            deployed_model_display_name="phase3-sklearn-deployed",
            dedicated_resources_machine_type="n1-standard-2",
            dedicated_resources_min_replica_count=1,
            dedicated_resources_max_replica_count=1,
            traffic_split={"0": "100"},
        )

In [7]:
# Compile and Run Pipeline

PIPELINE_YAML = "phase3_custom_container.yaml"
compiler.Compiler().compile(phase3_pipeline, PIPELINE_YAML)

job = aiplatform.PipelineJob(
    display_name=f"phase3-custom-container-{int(time.time())}",
    template_path=PIPELINE_YAML,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"n_rows": 500, "min_accuracy": 0.80},
    enable_caching=False,
)

job.run(service_account=SERVICE_ACCOUNT, sync=True)
print("State:", job.state)

Creating PipelineJob
PipelineJob created. Resource name: projects/208722280565/locations/us-central1/pipelineJobs/phase3-custom-container-single-endpoint-20260220215123
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/208722280565/locations/us-central1/pipelineJobs/phase3-custom-container-single-endpoint-20260220215123')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/phase3-custom-container-single-endpoint-20260220215123?project=208722280565
PipelineJob projects/208722280565/locations/us-central1/pipelineJobs/phase3-custom-container-single-endpoint-20260220215123 current state:
PipelineState.PIPELINE_STATE_PENDING
PipelineJob projects/208722280565/locations/us-central1/pipelineJobs/phase3-custom-container-single-endpoint-20260220215123 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/208722280565/locations/us-central1/pipelineJobs/phase3-custom-container-single-end

RuntimeError: Job failed with:
code: 9
message: "  The DAG failed because some tasks failed. The failed tasks are: [condition-1].; Job (project_id = vertex-ai-487907, job_id = 657893194796630016) is failed due to the above error.; Failed to handle the job: {project_number = 208722280565, job_id = 657893194796630016}"
